**Authors:** Anowar Shajib, David Law

*Adapted from the [notebook](https://github.com/JWST-Templates/Notebooks/blob/main/MIRI_MRS_reduction_SPT0418-47_PAH_ch3long.ipynb) by the TEMPLATES team (J. Spilker, K. A. Phadke, D. Law).*

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from scipy.ndimage import median_filter
import pathlib

from util import *

In [ ]:
run_pipeline = False
do_extra_processing = True
rate_file_directory_for_processing = stage1_directory

msa_leakage_file_ids = ["10101", "12101", "14101", "16101"]

# STAGE 1 DETECTOR PIPELINE

In [ ]:
# Define a function to call the detector1 pipeline with desired parameters.
# Individual steps not enumerated. Options can be set as commented overrides.


def run_spec1_pipeline(file_name, output_directory):
    """
    Create a Detector1Pipeline object and set all the desired stage 1
    pipeline processing arguments.

    :param file_name: _uncal.fits exposure file name
    :param output_directory: Directory for the stage 1 output files
    :return: None
    :outputs: writes  _rate.fits and _rateints.fits files in `outdir`
    """

    print("Running Detector1Pipeline on {0:s}...".format(file_name))

    crds_config = Detector1Pipeline.get_config_from_reference(file_name)
    detector_1_pipeline = Detector1Pipeline.from_config_section(crds_config)
    detector_1_pipeline.output_dir = output_directory

    # Overrides for whether or not certain steps should be skipped
    # detector_1_pipeline.dq_init.skip = False
    # detector_1_pipeline.saturation.skip = False
    # detector_1_pipeline.firstframe.skip = False
    # detector_1_pipeline.lastframe.skip = False
    # detector_1_pipeline.reset.skip = False
    # detector_1_pipeline.linearity.skip = False
    # detector_1_pipeline.rscd.skip = False
    # detector_1_pipeline.dark_current.skip = False
    # detector_1_pipeline.refpix.skip = False
    # detector_1_pipeline.jump.skip = False
    # detector_1_pipeline.ramp_fit.skip = False
    # detector_1_pipeline.gain_scale.skip = False

    # Trun on multiprocessing for jump and ramp fitting steps
    detector_1_pipeline.jump.maximum_cores = "half"
    detector_1_pipeline.ramp_fit.maximum_cores = "half"

    # S_DARK  = 'COMPLETE'           / Dark Subtraction
    # S_DQINIT= 'COMPLETE'           / Data Quality Initialization
    # S_GANSCL= 'COMPLETE'           / Gain Scale Correction
    # S_GRPSCL= 'COMPLETE'           / Group Scale Correction
    # S_IPC   = 'SKIPPED '           / Interpixel Capacitance Correction
    # S_JUMP  = 'COMPLETE'           / Jump Detection
    # S_LINEAR= 'COMPLETE'           / Linearity Correction
    # S_RAMP  = 'COMPLETE'           / Ramp Fitting
    # S_REFPIX= 'COMPLETE'           / Reference Pixel Correction
    # S_SATURA= 'COMPLETE'           / Saturation Checking
    # S_SUPERB= 'COMPLETE'           / Superbias Subtraction

    # Bad pixel mask overrides
    # detector_1_pipeline.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    # et1.saturation.override_saturation = 'myfile.fits'

    # Reset overrides
    # detector_1_pipeline.reset.override_reset = 'myfile.fits'

    # Linearity overrides
    # detector_1_pipeline.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    # detector_1_pipeline.rscd.override_rscd = 'myfile.fits'

    # DARK overrides
    # detector_1_pipeline.dark_current.override_dark = 'myfile.fits'

    # GAIN overrides
    # detector_1_pipeline.jump.override_gain = 'myfile.fits'
    # detector_1_pipeline.ramp_fit.override_gain = 'myfile.fits'

    # READNOISE overrides
    # detector_1_pipeline.jump.override_readnoise = 'myfile.fits'
    # detector_1_pipeline.ramp_fit.override_readnoise = 'myfile.fits'

    # JUMP overrides.
    # Currently pipeline is not flagging lower-level jumps
    # like we might want it to, so lower thresholds for more
    # aggressive flagging.
    # detector_1_pipeline.jump.save_results = True
    # detector_1_pipeline.jump.rejection_threshold = 3.5  # default 4.0sigma
    # detector_1_pipeline.jump.min_jump_to_flag_neighbors = 8.0  # default 10sigma

    # Additional JUMP overrides related to CR shower flagging. See
    # JWST pipeline documentation page for details of parameters.
    # https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html
    detector_1_pipeline.jump.expand_large_events = True  # Turn on shower flagging
    # detector_1_pipeline.jump.use_ellipses = True  # approximate showers as elliptical, obsolete in newest pipeline
    # detector_1_pipeline.jump.min_jump_area = 12  # Min # of contiguous pixels to trigger expanded flagging
    # detector_1_pipeline.jump.sat_required_snowball = False  # Require pixels to be saturated to flag
    # detector_1_pipeline.jump.expand_factor = 3.0  # expands showers beyond ID'd jumps; default 2.0
    #
    # detector_1_pipeline.jump.after_jump_flag_dn1 = 10  # These 4 related to how long after a jump is identified
    # detector_1_pipeline.jump.after_jump_flag_time1 = 20  #  we should keep flagging the following integrations
    # detector_1_pipeline.jump.after_jump_flag_dn2 = 1000
    # detector_1_pipeline.jump.after_jump_flag_time2 = 3000

    ### settings from the original TEMPLATES notebook used for MIRI, for reference
    # # JUMP overrides.
    # # Currently pipeline is not flagging lower-level jumps
    # # like we might want it to, so lower thresholds for more
    # # aggressive flagging.
    # #detector_1_pipeline.jump.save_results = True
    # detector_1_pipeline.jump.rejection_threshold = 3.5         # default 4.0sigma
    # detector_1_pipeline.jump.min_jump_to_flag_neighbors = 8.0  # default 10sigma
    #
    # # Additional JUMP overrides related to CR shower flagging. See
    # # JWST pipeline documentation page for details of parameters.
    # # https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html
    # detector_1_pipeline.jump.expand_large_events   = True   # Turn on shower flagging
    # detector_1_pipeline.jump.use_ellipses          = True   # True for MIRI; approximate showers as elliptical
    # detector_1_pipeline.jump.min_jump_area         = 8     # Min # of contiguous pixels to trigger expanded flagging
    # # The saturated core allows the search for smaller events without false positives.
    # detector_1_pipeline.jump.sat_required_snowball = False  # Do not require pixels to be saturated to flag
    # detector_1_pipeline.jump.expand_factor         = 3.0    # expands showers beyond ID'd jumps; default 2.0
    #
    # detector_1_pipeline.jump.after_jump_flag_dn1   = 10     # These 4 related to how long after a jump is identified
    # detector_1_pipeline.jump.after_jump_flag_time1 = 20     #  we should keep flagging the following integrations
    # detector_1_pipeline.jump.after_jump_flag_dn2   = 1000
    # detector_1_pipeline.jump.after_jump_flag_time2 = 3000
    ### TEMPLATES settings end

    # Ramp_fit overrides.
    # detector_1_pipeline.ramp_fit.save_opt = True
    # detector_1_pipeline.ramp_fit.save_results = True

    detector_1_pipeline.save_results = True  # Save the final resulting _rate.fits files

    detector_1_pipeline(file_name)  # Run the pipeline on an input list of files

In [ ]:
# grab a list of all uncal files from the folder
files_uncal = sorted(glob.glob(uncal_directory + "*nrs1_uncal.fits"))

# Run the Detector1 pipeline on each uncal exposure.
if run_pipeline:
    for file in tqdm(files_uncal):
        run_spec1_pipeline(file_name=file, output_directory=stage1_directory)

# Extra processing to subtract the leak images

In [ ]:
if do_extra_processing:
    # Make a median leak image
    all_files = np.array(sorted(glob.glob(rate_file_directory_for_processing + "*nrs1_rate.fits")))
    leak_files = []
    for file in all_files:
        for msa_leakage_file_id in msa_leakage_file_ids:
            if msa_leakage_file_id in file:
                leak_files.append(file)
                break

    # Read in all the leak files
    big_array = np.zeros([len(leak_files), 2048, 2048])
    for i, leak_file in enumerate(leak_files):
        with fits.open(leak_file) as hdu:
            big_array[i, :, :] = hdu["SCI"].data

    median_leak = np.nanmedian(big_array, axis=0)

    # Clean up the leak image
    leak2 = median_leak.copy()

    # First mask anything with counts > 0.5 DN/s
    leak2[median_leak > 0.5] = np.nanmedian(leak2)
    
    # Then deal with NaNs
    leak2[np.isnan(leak2)] = np.nanmedian(leak2)

    # Then apply a horizontal median filter to take out RMS noise
    leak2 = median_filter(leak2, size=(1, 51))

    # Subtract leak image from science files
    sci_files = []
    for file in all_files:
        if file not in leak_files:
            sci_files.append(file)

    for sci_file in sci_files:
        with fits.open(sci_file) as hdu:
            sci = hdu["SCI"].data
            sci = sci - leak2
            hdu["SCI"].data = sci
            name = pathlib.Path(sci_file).name
            hdu.writeto(stage1_processed_directory + name, overwrite=True)

# Extra processing to flag bad pixels

In [ ]:
if do_extra_processing:
    files = np.array(sorted(glob.glob(stage1_processed_directory + "*1001*nrs1_rate.fits")))

    for file in files:
        with fits.open(file) as hdu:
            sci = hdu["SCI"].data

            # Flag positive pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 2
            index = np.where(diff > 0.5)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)

            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            # Flag negative pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 0.5
            index = np.where(diff < -0.5)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)
            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            hdu["SCI"].data = sci
            hdu.writeto(file, overwrite=True)